In [2]:
import os
import shutil
from pathlib import Path
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [10]:
core='WDC'

df = pd.read_csv(f'/Users/quinnmackay/Documents/GitHub/BICC/Paleochrono BICC Work/Paleochrono BICC Experiment/BICC2025/{core}/deposition.txt', comment='#', sep='\t')
df['rel_unc'] = 0.2
df.to_csv(f'/Users/quinnmackay/Desktop/temp/export_{core}.txt')

In [6]:
print(df['dur_unc'].max())

44.700000000000045
